<a href="https://colab.research.google.com/github/pndang/USP_138_Project/blob/main/USP_138_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

import warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive


In [76]:
columns = ['2017 NAICS code (NAICS2017)',
           'Meaning of NAICS code (NAICS2017_LABEL)',
           'Meaning of Employment size of establishments code (EMPSZES_LABEL)',
           'Number of employees (EMP)']

df = pd.read_csv("/content/drive/MyDrive/USP_138/SF_Industries.csv", \
                 usecols=columns)

df_3d = pd.read_csv("/content/drive/MyDrive/USP_138/SF_manufacturing_3digitsNAICS.csv", \
                    usecols=columns)

us_3d = pd.read_csv("/content/drive/MyDrive/USP_138/US_manufacturing_3digitsNAICS.csv", \
                    usecols=['Meaning of Legal form of organization code (LFO_LABEL)']+columns)

df.head()

,2017 NAICS code (NAICS2017),Meaning of NAICS code (NAICS2017_LABEL),Meaning of Employment size of establishments code (EMPSZES_LABEL),Number of employees (EMP)
0,00,Total for all sectors,All establishments,"720,508"
1,00,Total for all sectors,Establishments with less than 5 employees,N
2,00,Total for all sectors,Establishments with 5 to 9 employees,N
3,00,Total for all sectors,Establishments with 10 to 19 employees,N
4,00,Total for all sectors,Establishments with 20 to 49 employees,N


In [77]:
df_3d.head()

,2017 NAICS code (NAICS2017),Meaning of NAICS code (NAICS2017_LABEL),Meaning of Employment size of establishments code (EMPSZES_LABEL),Number of employees (EMP)
0,311,Food manufacturing,All establishments,"2,052"
1,311,Food manufacturing,Establishments with less than 5 employees,N
2,311,Food manufacturing,Establishments with 5 to 9 employees,N
3,311,Food manufacturing,Establishments with 10 to 19 employees,N
4,311,Food manufacturing,Establishments with 20 to 49 employees,N


In [78]:
us_3d.head()

,2017 NAICS code (NAICS2017),Meaning of NAICS code (NAICS2017_LABEL),Meaning of Legal form of organization code (LFO_LABEL),Meaning of Employment size of establishments code (EMPSZES_LABEL),Number of employees (EMP)
0,311,Food manufacturing,All establishments,All establishments,"1,614,185"
1,311,Food manufacturing,All establishments,Establishments with less than 5 employees,"17,375"
2,311,Food manufacturing,All establishments,Establishments with 5 to 9 employees,"32,331"
3,311,Food manufacturing,All establishments,Establishments with 10 to 19 employees,"61,131"
4,311,Food manufacturing,All establishments,Establishments with 20 to 49 employees,"123,949"


In [79]:
rename_dict = {'2017 NAICS code (NAICS2017)': 'NAICS Code',
                   'Meaning of NAICS code (NAICS2017_LABEL)': 'Industry',
                   'Number of employees (EMP)': 'Number of Employees'}

df.rename(columns=rename_dict, inplace=True)
df_3d.rename(columns=rename_dict, inplace=True)
us_3d.rename(columns=rename_dict, inplace=True)

In [80]:
# df_3d is structured slight different from df, and the total across all industries must be manually calculated

test = df_3d[df_3d['Meaning of Employment size of establishments code (EMPSZES_LABEL)'] == 'All establishments']
test['Number of Employees'] = test['Number of Employees'].str.replace(',', '').astype(float)
test['Number of Employees'].sum()

7723.0

In [81]:
dummy_df = pd.DataFrame(data={'NAICS Code': ['___'], 'Industry': ['Total'], \
                              'Meaning of Employment size of establishments code (EMPSZES_LABEL)': ['All establishments'],
                              'Number of Employees': ['7,723']})

df_3d = pd.concat([dummy_df, df_3d], ignore_index=True)

In [82]:
def adjust_format_noe(row):
  noe = str(row['Number of Employees'])
  noe = noe[::-1]
  output = ''

  for i in range(0, len(noe), 3):
    output = output+','+noe[i:i+3]

  return output[::-1].strip(',')


In [83]:
# Pipeline to with steps to clean data

def clean_pl(data):
  data = data[data['Meaning of Employment size of establishments code (EMPSZES_LABEL)'] == 'All establishments']
  data.drop(columns=[x for x in data.columns if 'Meaning' in x], inplace=True)

  data['Number of Employees'] = data['Number of Employees'].str.replace(',','').astype(int)

  total = data['Number of Employees'][0]
  data['Percent'] = (data['Number of Employees'] / total)*100

  data['Percent'] = np.round(data['Percent'], 2)

  data.set_index('NAICS Code', inplace=True)

  data.sort_values(by='Percent', ascending=False, inplace=True)

  data['Number of Employees'] = data.apply(adjust_format_noe, axis=1)

  data['Percent'] = data['Percent'].astype(str)+'%'
  return data

In [84]:
df = clean_pl(df)
df_3d = clean_pl(df_3d)

In [85]:
pd.set_option('display.max_colwidth', None)

In [86]:
df

,Industry,Number of Employees,Percent
NAICS Code,,,
00,Total for all sectors,"720,508",100.0%
54,"Professional, scientific, and technical services","134,173",18.62%
51,Information,"87,285",12.11%
72,Accommodation and food services,"86,099",11.95%
62,Health care and social assistance,"74,186",10.3%
52,Finance and insurance,"64,272",8.92%
44-45,Retail trade,"49,806",6.91%
56,Administrative and support and waste management and remediation services,"41,224",5.72%
55,Management of companies and enterprises,"28,879",4.01%


In [87]:
df_3d.iloc[:16]

,Industry,Number of Employees,Percent
NAICS Code,,,
___,Total,"7,723",100.0%
311,Food manufacturing,"2,052",26.57%
334,Computer and electronic product manufacturing,"1,231",15.94%
312,Beverage and tobacco product manufacturing,813,10.53%
323,Printing and related support activities,724,9.37%
315,Apparel manufacturing,483,6.25%
339,Miscellaneous manufacturing,426,5.52%
327,Nonmetallic mineral product manufacturing,318,4.12%
337,Furniture and related product manufacturing,308,3.99%


### Calculate Location Quotient for Top 5 manufacturing industries

In [104]:
us_total = 134163349
sf_total = 720508

In [89]:
us_3d = us_3d[(us_3d['Meaning of Legal form of organization code (LFO_LABEL)'] == 'All establishments') & \
              (us_3d['Meaning of Employment size of establishments code (EMPSZES_LABEL)'] == 'All establishments')]
us_3d.drop(columns=['Meaning of Legal form of organization code (LFO_LABEL)', 'Meaning of Employment size of establishments code (EMPSZES_LABEL)'], inplace=True)

In [90]:
us_3d.head()

,NAICS Code,Industry,Number of Employees
0,311,Food manufacturing,"1,614,185"
63,312,Beverage and tobacco product manufacturing,"267,982"
117,313,Textile mills,"86,023"
157,314,Textile product mills,"107,626"
203,315,Apparel manufacturing,"74,387"


In [111]:
def calc_location_quotient(row):

  industry = row['Industry']
  sf_num = float(df_3d[df_3d['Industry'] == industry]['Number of Employees'].iloc[0].replace(',', ''))
  us_num = float(us_3d[us_3d['Industry'] == industry]['Number of Employees'].iloc[0].replace(',', ''))

  return np.round((sf_num / sf_total) / (us_num / us_total),2)


In [116]:
top5 = df_3d[1:].copy()
top5['Location Quotient'] = top5.apply(calc_location_quotient, axis=1)
top5.sort_values(by='Location Quotient', ascending=False, inplace=True)
top5.drop(columns=['Number of Employees', 'Percent'], inplace=True)

In [117]:
top5.head()

,Industry,Location Quotient
NAICS Code,,
315,Apparel manufacturing,1.21
312,Beverage and tobacco product manufacturing,0.56
316,Leather and allied product manufacturing,0.55
323,Printing and related support activities,0.33
334,Computer and electronic product manufacturing,0.29
